# MLP Clustering Significance Stability over Multiple Trainings - Different #Clusters

### Before Running on Perceptron

Go to `src/train_nn.py` to line 46, and change the path `./models/` to `/scratch/<your username>/models`

Do not forget to undo this change!

DO NOT COMMIT THIS CHANGE TO GITHUB!!!

In [ ]:
N_CLUSTERS = 4

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
import sys
sys.path.append('..')

import random
import subprocess
from multiprocessing import Pool

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from tqdm import tqdm, trange
from IPython import display

from src.visualization import draw_mlp_clustering_report, run_double_spectral_cluster, run_spectral_cluster
from src.utils import get_weights_paths, build_clustering_results, cohen_d_stats
from src.experiment_tagging import get_model_path, MODEL_TAG_LOOKUP
# from src.spectral_cluster_model import SHUFFLE_METHODS
SHUFFLE_METHODS = ['layer', 'layer_nonzero']

In [ ]:
N_TRAINED_MODELS = 10

# we ar doing 320 experiments (=32 models x 10 times),
# and let's have expectation of false-positive <= 1
# so we can set n_shuffles to 320, and then p-values < 1/401
# if we make it too large, let see, 1000 shuffles...
# ... we might reduce the statistical power (given H1, the probability to reject it incorrectly)!
N_SHUFFLES = 320

N_GPUS = 4

TAGS_COMMANDS = {
    'MNIST': 'python -m src.train_nn with mlp_config dataset_name=mnist',
    'CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=cifar10 pruning_epochs=40',
#    'LINE': 'python -m src.train_nn with mlp_config dataset_name=line',
    'FASHION': 'python -m src.train_nn with mlp_config dataset_name=fashion',
    'MNIST+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist with_dropout=True',
    'CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=cifar10 epochs=100 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'LINE+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line with_dropout=True',
    'FASHION+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=fashion with_dropout=True',
#    'LINE-MNIST': 'python -m src.train_nn with mlp_config dataset_name=line-mnist',
#    'LINE-CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10 epochs=30 pruning_epochs=40',
#    'MNIST-CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10 epochs=30 pruning_epochs=40',
#    'LINE-MNIST-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=line-mnist-separated',
#    'LINE-CIFAR10-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10-separated epochs=30 pruning_epochs=40',
#    'MNIST-CIFAR10-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10-separated epochs=30 pruning_epochs=40',
#    'LINE-MNIST+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-mnist with_dropout=True',
#    'LINE-CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10 epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'MNIST-CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10 epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'LINE-MNIST-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-mnist-separated with_dropout=True',
#    'LINE-CIFAR10-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10-separated epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'MNIST-CIFAR10-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10-separated epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'RANDOM': 'python -m src.train_nn with mlp_config dataset_name=random',
#    'RANDOM+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random with_dropout=True',
#    'MNIST-x1.5-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=30',
#    'MNIST-x1.5-EPOCHS+DROPOUT':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=30 with_dropout=True',
#    'MNIST-x2-EPOCHS':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=40',
#    'MNIST-x2-EPOCHS+DROPOUT':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=40 with_dropout=True',
#    'MNIST-x10-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=200',
#    'MNIST-x10-EPOCHS+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=200 with_dropout=True',
#    'RANDOM-x50-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=random epochs=1000',
#    'RANDOM-x50-EPOCHS+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random epochs=1000 with_dropout=True',
#    'RANDOM-OVERFITTING': 'python -m src.train_nn with mlp_config dataset_name=random epochs=100 pruning_epochs=100 shuffle=False n_train=3000',
#    'RANDOM-OVERFITTING+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random epochs=100 pruning_epochs=100 shuffle=False n_train=3000 with_dropout=True'
}
DATASETS_TAGS = [command.split()[5][13:] for command in TAGS_COMMANDS.values()]

In [ ]:
if False:
    def train_model_multiply(command, n_models_per_command, gpu_id=None):    
        for _ in range(n_models_per_command):
            actual_command = f'cd .. && {command}'#' > /dev/null 2>&1'
            print(actual_command)
            subprocess.run(actual_command, shell=True,
                           env={'CUDA_VISIBLE_DEVICES': str(gpu_id)})


    def train_models(commands, n_models_per_command, gpu_id=None):
        for command in commands:
            train_model_multiply(command, n_models_per_command, gpu_id)

            commands = list(TAGS_COMMANDS.values())
    random.shuffle(commands)

    assert len(commands) % N_GPUS == 0
    n_commands_per_gpu = len(commands) // N_GPUS

    with Pool(N_GPUS) as p:

        p.starmap(train_models,
                 [(commands[gpu_id*n_commands_per_gpu : (gpu_id+1)*n_commands_per_gpu], N_TRAINED_MODELS, gpu_id)
                 for gpu_id in range(N_GPUS)])

In [ ]:
results = {}

for shuffle_method in SHUFFLE_METHODS:
    results[shuffle_method] = {}
    
    for (model_tag, _), dataset_tag in zip(TAGS_COMMANDS.items(), tqdm(DATASETS_TAGS)):
        model_paths = get_model_path(model_tag, filter_='all', model_base_path='/scratch/shlomi/models/')


        results[shuffle_method][model_tag] = [run_spectral_cluster(path / f'{dataset_tag}-mlp-pruned-weights.pckl',
                                               n_samples=N_SHUFFLES,
                                               shuffle_method=shuffle_method,
                                               n_clusters=N_CLUSTERS)
            for path in tqdm(model_paths[-N_TRAINED_MODELS:])]

In [ ]:
run_spectral_cluster?

In [ ]:
def build_all_models_per_shuffle_method_df(single_shuffle_method_results):
    model_dfs = []

    for model_tag, model_results in single_shuffle_method_results.items():
        _, metrics  = zip(*model_results)
        model_dfs.append(pd.DataFrame(metrics).assign(model=model_tag))

    df = pd.concat(model_dfs, ignore_index=True)

    df['is_sig'] = np.isclose(df['percentile'], 1 / (N_SHUFFLES + 1))
    
    return df

layer_df = build_all_models_per_shuffle_method_df(results['layer'])
layer_nonzero_df = build_all_models_per_shuffle_method_df(results['layer_nonzero'])

In [ ]:
layer_df.head()

In [ ]:
layer_nonzero_df.head()

In [ ]:
MODEL_DATA_COLUMNS = ['model',
                      'train_acc', 'train_loss', 'test_acc', 'test_loss',
                      'ncut', 'ave_in_out',
                      'n_samples']

assert layer_df[MODEL_DATA_COLUMNS].equals(layer_nonzero_df[MODEL_DATA_COLUMNS])

SHUFFLING_DATA_COLUMNS = ['mean', 'percentile', 'stdev', 'z_score', 'is_sig']

two_shuffling_method_df = pd.merge(layer_df[SHUFFLING_DATA_COLUMNS],
                                     layer_nonzero_df[SHUFFLING_DATA_COLUMNS],
                                     left_index=True, right_index=True,
                                     suffixes=('_layer', '_layer_nonzero'))

assert layer_df[MODEL_DATA_COLUMNS].equals(layer_nonzero_df[MODEL_DATA_COLUMNS])

df = pd.merge(layer_df[MODEL_DATA_COLUMNS], two_shuffling_method_df,
              left_index=True, right_index=True,)

df['cohen_d'] = df.apply(lambda r: cohen_d_stats(r['mean_layer'], r['stdev_layer'], r['n_samples'],
                                 r['mean_layer_nonzero'], r['stdev_layer_nonzero'], r['n_samples']),
                                axis=1)

df

In [ ]:
# df = pd.read_csv('../results/mlp-clustering-stability-two-shuffling-methods-all-samples.csv' index_col=0)

In [ ]:
df_grpby_model = df.groupby('model')

TWO_SHUFFLE_STATS_COLUMNS = [f'{stat}_{shuffle_method}'
                             for stat in ['mean', 'stdev', 'z_score', 'percentile', 'is_sig']
                             for shuffle_method in SHUFFLE_METHODS]

statistics_df = pd.concat([(df_grpby_model[['ncut'] + TWO_SHUFFLE_STATS_COLUMNS]
                             .agg(['mean', 'std'])),
                            df_grpby_model.size().rename('n_models'),
                            (df_grpby_model[['train_acc', 'test_acc']]
                             .agg('mean'))],
                           axis=1)

In [ ]:
statistics_df

In [ ]:
# TODO: ask for a code review
# std = np.sqrt(np.mean(x**2) - np.mean(x)**2)

def calc_overall_stdev(row, suffix=''):
    return np.sqrt(                 # 5. apply root square to get the SD from the variance
        (np.mean(                   # 3. divide by the number of trained model
            (row[f'stdev{suffix}']**2)       # 1. cancel the overall square root
             + row[f'mean{suffix}']**2))     # 2. add the mean and get the sum or squares
        - np.mean(row[f'mean{suffix}'])**2)  # 4. substrct the square of the overall mean

In [ ]:
SHUFFLE_METHOD_SUFFIX = ['_layer', '_layer_nonzero']
for suffix in SHUFFLE_METHOD_SUFFIX:
    statistics_df[f'overall_std{suffix}'] = df_grpby_model.apply(calc_overall_stdev, suffix)

for suffix in SHUFFLE_METHOD_SUFFIX:
    statistics_df[f'is_stable{suffix}'] = (statistics_df[(f'is_sig{suffix}', 'mean')] == 1)

statistics_df = statistics_df.loc[TAGS_COMMANDS.keys(), :]

In [ ]:
statistics_df

In [ ]:
df.to_csv(f'../results/mlp-clustering-stability-two-shuffling-methods-all-samples-K-{N_CLUSTERS}.csv')
statistics_df.to_csv(f'../results/mlp-clustering-stability-two-shuffling-methods-statistics-K-{N_CLUSTERS}.csv')

### Model Stability Statistics

In [ ]:
statistics_df

### Appendix - All Samples

In [ ]:
df

### Unpruned results without shuffling
#### TODO: Refactor
- [ ] Combine with the unpruned table

In [ ]:
if False:
    unpruned_results = {}

    for (model_tag, _), dataset_tag in zip(TAGS_COMMANDS.items(), tqdm(DATASETS_TAGS)):
        model_paths = get_model_path(model_tag, filter_='all', model_base_path='/scratch/shlomi/models/')
        unpruned_results[model_tag] = [run_spectral_cluster(path / f'{dataset_tag}-mlp-unpruned-weights.pckl',
                                                   with_shuffle=False)
                for path in tqdm(model_paths[-N_TRAINED_MODELS:])]

    unpruned_model_dfs = []

    for model_tag, model_results in unpruned_results.items():
        _, metrics  = zip(*model_results)
        unpruned_model_dfs.append(pd.DataFrame(metrics).assign(model=model_tag))

    unpruned_df = pd.concat(unpruned_model_dfs, ignore_index=True)

    unpruned_df_grpby_model = unpruned_df.groupby('model')


    unpruned_statistics_df = pd.concat([(unpruned_df_grpby_model[['ncut']]
                                 .agg(['mean', 'std'])),
                                unpruned_df_grpby_model.size().rename('n_models'),
                                (unpruned_df_grpby_model[['train_acc', 'test_acc']]
                                 .agg('mean'))],
                               axis=1)

    unpruned_statistics_df = unpruned_statistics_df.loc[TAGS_COMMANDS.keys(), :]

    unpruned_statistics_df

In [ ]:
# unpruned_df.to_csv(f'../results/mlp-clustering-stability-all-samples-unpruned-K-{N_CLUSTERS}.csv')
# unpruned_statistics_df.to_csv(f'../results/mlp-clustering-stability-statistics-unpruned-K-{N_CLUSTERS}.csv')